<a href="https://colab.research.google.com/github/staerkjoe/QuantRisk_Colab/blob/main/TicketCancellation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("pkdarabi/classification-of-travel-purpose")

print("Path to dataset files:", path)

100%|██████████| 7.30M/7.30M [00:00<00:00, 116MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/pkdarabi/classification-of-travel-purpose/versions/1


Link to dataset:
https://www.kaggle.com/datasets/pkdarabi/classification-of-travel-purpose

In [4]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 7.3 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
import os
import time
from unidecode import unidecode

In [6]:
for file in os.listdir(path):
    print(file)

train_data.csv


In [7]:
df = pd.read_csv(os.path.join(path, "train_data.csv"))

In [8]:
# drop columns that are unnecessary/not helpful for prediction

# List of columns to drop
drop_cols = [
    'BillID',               # Unique identifier
    'TicketID',             # Unique identifier
    'UserID',               # Sparse and semi-unique
    'BuyerMobile',          # Personal identifier
    'NationalCode',         # Personal identifier
    'HashPassportNumber_p', # Sparse and hashed
    'HashEmail',            # Sparse and hashed
    'CancelTime',           # Leaks target label
    'VehicleType'
]

# Drop columns from DataFrame
df = df.drop(columns=drop_cols)

# Optional: Confirm result
print("Remaining columns:", df.columns.tolist())




Remaining columns: ['Created', 'DepartureTime', 'ReserveStatus', 'Male', 'Price', 'CouponDiscount', 'From', 'To', 'Domestic', 'VehicleClass', 'TripReason', 'Vehicle', 'Cancel']


In [9]:
# Drop Nulla values for From and TO
df = df.dropna(subset=['From', 'To'])

# Combine 'From' and 'To' columns to create a list of all cities
all_cities = df['From'].tolist() + df['To'].tolist()

# Create a set of unique cities and convert it back to a list
unique_cities = list(set(all_cities))

# print(unique_cities)

# Mapping erstellen
city_mapping = {city: unidecode(city) for city in unique_cities}

'''
# Ausgabe als Dictionary
print("city_mapping = {")
for k, v in city_mapping.items():
    print(f"    '{k}': '{v}',")
print("}")
'''

'\n# Ausgabe als Dictionary\nprint("city_mapping = {")\nfor k, v in city_mapping.items():\n    print(f"    \'{k}\': \'{v}\',")\nprint("}")\n'

In [10]:
def normalize_city_name(name):
    if pd.isna(name):
        return None
    return (
        str(name)
        .strip()
        .replace('\u200c', '')  # remove zero-width non-joiner
        .replace(' (', '(')
        .replace(')', ')')
        .replace('‌', '')       # remove zero-width joiner
    )

def safe_unidecode(value):
    if pd.isna(value):
        return ""
    return unidecode(str(value))

df['From'] = df['From'].apply(normalize_city_name).map(city_mapping).apply(safe_unidecode)
df['To'] = df['To'].apply(normalize_city_name).map(city_mapping).apply(safe_unidecode)

df['FromMapped'] = df['From'].isin(city_mapping.values())
df['ToMapped'] = df['To'].isin(city_mapping.values())

# keep only rows where both 'From' and 'To' were mapped
df = df[df["FromMapped"] & df["ToMapped"]].copy()
# drop mapping flags
df.drop(columns=["FromMapped", "ToMapped"], inplace=True)

In [11]:
df['Price'].describe()

,Price
count,9.167200e+04
mean,3.333467e+06
std,7.696764e+06
min,-1.514000e+06
25%,1.025000e+06
50%,1.680000e+06
75%,3.150000e+06
max,3.837646e+08


In [12]:
q25 = df['Price'].quantile(0.25)
q75 = df['Price'].quantile(0.75)

df = df[(df['Price'] >= q25) & (df['Price'] <= q75)].copy()



In [13]:
# Columns Transformations

# Date FOrmat
df[['Created', 'DepartureTime']] = df[['Created', 'DepartureTime']].apply(pd.to_datetime)

# Price to €
exchange_rate = 0.00002027  # 1 IRR = 0.00002027 EUR
df['Price'] = (df['Price'] * exchange_rate).round(2)
df['CouponDiscount'] = (df['CouponDiscount'] * exchange_rate).round(2)


In [14]:
# Fearure Engineering
df['LeadTime'] = (df['DepartureTime'] - df['Created']).dt.total_seconds()
df['IsWeekend'] = (df['DepartureTime'].dt.weekday >= 5).astype(int)
df['DiscountRatio'] = np.where(df['Price'] > 0, df['CouponDiscount'] / df['Price'], 0)
df['BookingMonth'] = df['Created'].dt.month
df['DepartureMonth'] = df['DepartureTime'].dt.month
df['BookingWeekday'] = df['Created'].dt.weekday
df['DepartureWeekday'] = df['DepartureTime'].dt.weekday
df['DepartureHour'] = df['DepartureTime'].dt.hour
df['BookingHour'] = df['Created'].dt.hour
df["Male"] = df["Male"].astype(int)

# drop raw datetime columns
df.drop(columns=['Created', 'DepartureTime'], inplace=True)

In [15]:
# Rename the column
df.rename(columns={'VehicleClass': 'VIPStatus'}, inplace=True)

# Replace NaN values with 'Unknown'
df['VIPStatus'] = df['VIPStatus'].fillna('Unknown')
df["VIPStatus"] = df["VIPStatus"].astype(str)


In [16]:
df.head()

,ReserveStatus,Male,Price,CouponDiscount,From,To,Domestic,VIPStatus,TripReason,Vehicle,Cancel,LeadTime,IsWeekend,DiscountRatio,BookingMonth,DepartureMonth,BookingWeekday,DepartureWeekday,DepartureHour,BookingHour
2,3,0,40.54,0.0,shyrz,hwz,1,True,Work,Bus,0,63272.750,0,0.0,9,9,1,2,11,17
7,3,1,50.27,0.0,thrn,shyrz,1,True,Work,Bus,0,7215.157,0,0.0,7,7,3,3,16,13
8,3,1,24.73,0.0,thrn,Sfhn,1,True,Work,Bus,0,57761.133,0,0.0,10,10,6,0,2,9
9,3,1,26.35,0.0,thrn,tbryz,1,True,Int,Bus,0,312199.797,0,0.0,3,3,3,0,13,22
14,2,1,23.48,0.0,yzd,thrn,1,Unknown,Int,Train,0,1471280.147,0,0.0,9,9,1,4,20,20


In [17]:
# use of frequency encoding since there is a lot of different cities and other encodings would not make sense
# loss of interpretability though

df['FromFreq'] = df['From'].map(df['From'].value_counts())
df['ToFreq'] = df['To'].map(df['To'].value_counts())

# drop the city columns
df.drop(columns=['From', 'To'], inplace=True)

In [18]:
df['DiscountRatio'].describe()

,DiscountRatio
count,45901.000000
mean,0.001288
std,0.015095
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.113043


In [19]:
df['Vehicle'].value_counts()

,count
Vehicle,
Bus,26712
Train,19001
Plane,184
InternationalPlane,4


In [22]:
# Create a copy of the cleaned DataFrame
cleaned_data = df.copy()

In [23]:
# Save as CSV
cleaned_data.to_csv(
    '/content/drive/MyDrive/Colab Notebooks/QantRisk/cleaned_data.csv',
    index=False
)

In [ ]:
cleaned_data.head()

In [ ]:
cleaned_data['IsWeekend'].value_counts()

In [ ]:
cleaned_data['Cancel'].value_counts()

In [ ]:
cleaned_data.info()